In [56]:
import os
import numpy as np
from numpy import linalg as lin

import pandas as pd
from scipy.optimize import minimize

from shapely.geometry import Polygon

In [99]:
path = "/home/sourabh/Documents/TU-Berlin/Thesis"
dataPath = "Sytheticdata/ml-imagesynthesis/captures"

folderPath = os.path.join(path, dataPath, "Train/")

def polygon_area(points):  
    """Return the area of the polygon whose vertices are given by the
    sequence points.

    """
    area = 0
    q = points[-1]
    for p in points:  
        area += p[0] * q[1] - p[1] * q[0]
        q = p
    return area / 2



ADDError = []
ADIError = []
ACPDError = []
MCPDError = []
IOUs = []
for i in range(2, 100):
    file = "image_%05d_img" %i
    dataGT = pd.read_csv(os.path.join(folderPath, "{}-GT.txt".format(file)), header=None).to_numpy()
    dataPnP = pd.read_csv(os.path.join(folderPath, "{}-PnP.txt".format(file)), header=None).to_numpy()
    
    ADD = lin.norm(dataGT - dataPnP)
    ADI = np.min([lin.norm(dataGT[i] - dataPnP[i]) for i in range(0, dataGT.shape[0])])
    ACPD = np.mean([lin.norm(dataGT[i] - dataPnP[i]) for i in range(0, dataGT.shape[0])])
    MCPD = np.max([lin.norm(dataGT[i] - dataPnP[i]) for i in range(0, dataGT.shape[0])])

    p1 = Polygon(dataGT)
    p2 = Polygon(dataPnP)
    
    IOU = np.divide(p1.intersection(p2).area, p1.union(p2).area)
    
    ADDError.append(ADD)
    ADIError.append(ADI)
    ACPDError.append(ACPD)
    MCPDError.append(MCPD)
    IOUs.append(IOU)
    
print('ADD error {} : ADI error {} ACPD error {} MCPD {} Mean IOU {}'.format(np.mean(ADDError), np.mean(ADIError), np.min(ACPDError)\
                                                                 , np.min(MCPDError), np.mean(IOUs)))

    
    

ADD error 58.411638589820065 : ADI error 4.36416212151522 ACPD error 1.5318877250509808 MCPD 3.22361870729155 Mean IOU 0.7112730481270574


In [112]:
# COncatenate images

dataPath = "Result-pictures/Report"
folderPath = os.path.join(path, dataPath, "images/")

import sys
from PIL import Image

images = [Image.open(x) for x in [os.path.join(folderPath,'image39.png'), \
                                os.path.join(folderPath,'image40.png'), \
                                os.path.join(folderPath,'image41.png')]]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = min(heights)

new_im = Image.new('RGB', (total_width, max_height))

x_offset = 0
for i, im in enumerate(images):
    
    im.resize((total_width, max_height), Image.ANTIALIAS)
    im.save('text{}.png'.format(i))
    new_im.paste(im, (x_offset,0))
    x_offset += im.size[0]

print('saving image')
new_im.save('test.png')

saving image
